In [305]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
import re
warnings.filterwarnings('ignore')

In [306]:
dataset = pd.read_csv('loan.csv')
findMeanAndReplace_cols = pd.read_csv('findMeanAndReplace_cols.csv')
remap_cols = pd.read_csv('remap_cols.csv')
drop_cols = pd.read_csv('drop_col.csv')
replaceWithZero_cols = pd.read_csv('replaceWithZero_cols.csv')

In [307]:
dataset.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Basic Feature Removal
1. Remove any column that has a null value <<< Not ideal. Change it.
2. Convert all non-numeric columns to numeric representations.

In [308]:
#Delete Columns
def deleteColumns(df, col):
    df.pop(col)

In [309]:
for col in list(drop_cols["Columns"]):
    deleteColumns(dataset,col)

In [310]:
print(dataset.isna().sum())

id                          0
member_id                   0
loan_amnt                   0
funded_amnt                 0
funded_amnt_inv             0
term                        0
int_rate                    0
installment                 0
grade                       0
sub_grade                   0
emp_length              44825
home_ownership              0
annual_inc                  4
verification_status         0
loan_status                 0
pymnt_plan                  0
dti                         0
open_acc                   29
total_acc                  29
initial_list_status         0
out_prncp                   0
total_pymnt                 0
total_rec_prncp             0
total_rec_int               0
total_rec_late_fee          0
recoveries                  0
last_pymnt_amnt             0
application_type            0
annual_inc_joint       886868
dti_joint              886870
tot_coll_amt            70276
dtype: int64


In [311]:
#Calculate mean of a column and Replace Nulls
def findMeanAndReplaceNull(df, col):
    col_values = list(df[col]);
    nan_values = 0
    sum = 0
    #Checking nan values in each column list so that it is not included in sum
    for value in col_values:
        if(math.isnan(value)):
            nan_values = nan_values+1
            #print("Nan")
        else:
            sum = sum+value
    col_mean = sum/(len(col_values)-nan_values) 
    print("Col_length : " + str(len(col_values))+"\tCol Sum : "+str(sum)+"\tMean : "+str(col_mean)+"\tCol_name : "+col)
    if(math.isnan(col_mean)):
        print("*************************"+col+"*************************")
    if(nan_values == len(col_values)):
        print("*************************"+col+"*************************")
    df[col].fillna(col_mean, inplace=True) #Replacing nulls with mean

In [312]:

def findMeanAndReplaceNullInDataset(df,findMeanAndReplace_cols):
    for col in list(findMeanAndReplace_cols["Columns"]):
        findMeanAndReplaceNull(df,col)


In [313]:
findMeanAndReplaceNullInDataset(dataset,findMeanAndReplace_cols)
#findMeanAndReplaceNull(dataset, "annual_inc")

Col_length : 887379	Col Sum : 13093511950.0	Mean : 14755.26460508982	Col_name : loan_amnt
Col_length : 887379	Col Sum : 13081632625.0	Mean : 14741.877625005776	Col_name : funded_amnt
Col_length : 887379	Col Sum : 13046658141.926006	Mean : 14702.46438322972	Col_name : funded_amnt_inv
Col_length : 887379	Col Sum : 11754878.610034421	Mean : 13.246739679476775	Col_name : int_rate
Col_length : 887379	Col Sum : 387533607.7601214	Mean : 436.7171273605995	Col_name : installment
Col_length : 887379	Col Sum : 66577605689.21044	Mean : 75027.58776076681	Col_name : annual_inc
Col_length : 887379	Col Sum : 16112174.879999854	Mean : 18.157038739929448	Col_name : dti
Col_length : 887379	Col Sum : 10247534.0	Mean : 11.548469037020341	Col_name : open_acc
Col_length : 887379	Col Sum : 22421583.0	Mean : 25.26802614526399	Col_name : total_acc
Col_length : 887379	Col Sum : 7456756527.960241	Mean : 8403.124851906841	Col_name : out_prncp
Col_length : 887379	Col Sum : 6707544063.703547	Mean : 7558.826683641992

In [314]:
def replaceWithZeroMissingValues(df,replaceWithZero_cols):
    for col in list(replaceWithZero_cols["Columns"]):
        df[col].fillna(0, inplace=True)

In [315]:
replaceWithZeroMissingValues(dataset,replaceWithZero_cols)

In [316]:
# Function to remap non-numeric values to a number value.
def remap(df, col) :
  col_set = list(set(df[col]));  # Get the column specified, covert it to set so that all duplicates are removed
  col_set_map = {val: i+1 for i, val in enumerate(sorted(col_set))} # Enumerate over the sorted set and build a Mapping dictionary
  df[col].replace(col_set_map, inplace=True) # Replace the values with the provided map
  print("Remaped", col, "with", col_set_map); # Log
  

In [317]:
def remapColumns(df,remap_cols):
    for col in list(remap_cols["Columns"]):
        print(col)
        remap(df,col)

In [318]:
remapColumns(dataset,remap_cols)

term
Remaped term with {' 36 months': 1, ' 60 months': 2}
grade
Remaped grade with {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
sub_grade
Remaped sub_grade with {'A1': 1, 'A2': 2, 'A3': 3, 'A4': 4, 'A5': 5, 'B1': 6, 'B2': 7, 'B3': 8, 'B4': 9, 'B5': 10, 'C1': 11, 'C2': 12, 'C3': 13, 'C4': 14, 'C5': 15, 'D1': 16, 'D2': 17, 'D3': 18, 'D4': 19, 'D5': 20, 'E1': 21, 'E2': 22, 'E3': 23, 'E4': 24, 'E5': 25, 'F1': 26, 'F2': 27, 'F3': 28, 'F4': 29, 'F5': 30, 'G1': 31, 'G2': 32, 'G3': 33, 'G4': 34, 'G5': 35}
verification_status
Remaped verification_status with {'Not Verified': 1, 'Source Verified': 2, 'Verified': 3}
loan_status
Remaped loan_status with {'Charged Off': 1, 'Current': 2, 'Default': 3, 'Does not meet the credit policy. Status:Charged Off': 4, 'Does not meet the credit policy. Status:Fully Paid': 5, 'Fully Paid': 6, 'In Grace Period': 7, 'Issued': 8, 'Late (16-30 days)': 9, 'Late (31-120 days)': 10}
home_ownership
Remaped home_ownership with {'ANY': 1, 'MORTGAGE': 2, 'NONE

In [319]:
def getLengthOfEmployee(value):
    length = re.findall('\d+',str(value))
    if(length[0].isnumeric()==False):
        print("Not numeric : "+length[0])
        
    #print(length[0])
    return length[0]

In [320]:
def remapEmpLength(df):
    df['emp_length'].replace('', np.nan, inplace=True)
    df["emp_length"].fillna(0, inplace=True)
    df['emp_length'] = df['emp_length'].apply(getLengthOfEmployee)

In [321]:
remapEmpLength(dataset)

In [322]:
dataset.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,last_pymnt_amnt,application_type,annual_inc_joint,dti_joint,tot_coll_amt
0,1077501,1296599,5000.0,5000.0,4975.0,1,10.65,162.87,2,7,...,5861.071414,5000.00,861.07,0.00,0.00,171.62,1,0.0,0.0,0.0
1,1077430,1314167,2500.0,2500.0,2500.0,2,15.27,59.83,3,14,...,1008.710000,456.46,435.17,0.00,117.08,119.66,1,0.0,0.0,0.0
2,1077175,1313524,2400.0,2400.0,2400.0,1,15.96,84.33,3,15,...,3003.653644,2400.00,603.65,0.00,0.00,649.91,1,0.0,0.0,0.0
3,1076863,1277178,10000.0,10000.0,10000.0,1,13.49,339.31,3,11,...,12226.302212,10000.00,2209.33,16.97,0.00,357.48,1,0.0,0.0,0.0
4,1075358,1311748,3000.0,3000.0,3000.0,2,12.69,67.79,2,10,...,3242.170000,2233.10,1009.07,0.00,0.00,67.79,1,0.0,0.0,0.0


In [323]:
dataset.keys()

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'pymnt_plan', 'dti', 'open_acc', 'total_acc', 'initial_list_status',
       'out_prncp', 'total_pymnt', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'last_pymnt_amnt',
       'application_type', 'annual_inc_joint', 'dti_joint', 'tot_coll_amt'],
      dtype='object')

Need to confirm if any value in the data frame is of non-numeric type. This is because non-numeric types cannot be used in training

In [328]:
columnsWithStringType = []
for k in dataset.keys() :
  col = dataset[k];
  for v in col :
    if type(v) == type(""):
      columnsWithStringType.append(k)
      break;
# print(columnsWithStringType)      
# assert len(columnsWithStringType) == 0, "Some columns have string type."
print("All columns are number type.");

All columns are number type.


In [325]:
dataset.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,last_pymnt_amnt,application_type,annual_inc_joint,dti_joint,tot_coll_amt
0,1077501,1296599,5000.0,5000.0,4975.0,1,10.65,162.87,2,7,...,5861.071414,5000.00,861.07,0.00,0.00,171.62,1,0.0,0.0,0.0
1,1077430,1314167,2500.0,2500.0,2500.0,2,15.27,59.83,3,14,...,1008.710000,456.46,435.17,0.00,117.08,119.66,1,0.0,0.0,0.0
2,1077175,1313524,2400.0,2400.0,2400.0,1,15.96,84.33,3,15,...,3003.653644,2400.00,603.65,0.00,0.00,649.91,1,0.0,0.0,0.0
3,1076863,1277178,10000.0,10000.0,10000.0,1,13.49,339.31,3,11,...,12226.302212,10000.00,2209.33,16.97,0.00,357.48,1,0.0,0.0,0.0
4,1075358,1311748,3000.0,3000.0,3000.0,2,12.69,67.79,2,10,...,3242.170000,2233.10,1009.07,0.00,0.00,67.79,1,0.0,0.0,0.0


In [326]:
#path = "D:\Academic\Data Science\lc_data"
dataset.to_csv("cleaned_data_new.csv")   